# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 8. Evaluasi Beberapa Algoritma Klasifikasi

Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Arthurito Nehemia Gilbert Eduardo Luke Keintjem

NIM   : 225150207111047

Kelas : PPM TIF-A

### 1. Tentukan metode klasifikasi yang akan dievaluasi

Pada bagian ini pilih metode klasifikasi yang akan dievaluasi. Simpan model klasifikasi
dalam vector model. Hasil evaluasi ditampung pada vector result.

In [348]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []

### 2. Baca dataset

Ambil dataset yang akan dievaluasi. Untuk praktikum ini dievaluasi 3 dataset sebagai berikut

*   dataR2
*   iris data
*   breast cancer

Untuk praktek dataset dataR2 digunakan sebagai contoh sedangkan dua data berikutnya akan
digunakan sebagai latihan.

In [349]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv

--2024-05-08 06:44:12--  https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘dataR2.csv.20’

dataR2.csv.20           [ <=>                ]   7.49K  --.-KB/s    in 0s      

2024-05-08 06:44:12 (111 MB/s) - ‘dataR2.csv.20’ saved [7665]



### 3. Tampilkan dataset

In [350]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('dataR2.csv')
dataset

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1
...,...,...,...,...,...,...,...,...,...,...
111,45,26.850000,92,3.330,0.755688,54.6800,12.100000,10.96000,268.230,2
112,62,26.840000,100,4.530,1.117400,12.4500,21.420000,7.32000,330.160,2
113,65,32.050000,97,5.730,1.370998,61.4800,22.540000,10.33000,314.050,2
114,72,25.590000,82,2.820,0.570392,24.9600,33.750000,3.27000,392.460,2


### 4. Membuat Set Data Validasi

Tujuan evaluasi adalah memperkirakan keakuratan model terbaik pada data tak terlihat dengan mengevaluasinya pada data tak terlihat aktual. Evaluasi ini dilakukan dengan cara membagi dataset menjadi dua bagian yakni sebagian digunakan sebagai data latih dan sebagian yang lain digunakan sebagai data uji. Beberapa praktikum sebelumnya telah dipraktekkan dengan menggunakan **metode train_test_split : X_train, X_validation, Y_train, Y_validation =
train_test_split(X,y,test_size=0.20, random_state=1)** Sebelum membuat data latih dan data uji. Langkah pertama pisahkan antara data attribute dan data

label suatu dataset. Karena datasaet dataR2 terdiri dari 9 attribut dan 1 label. Maka kita catat X sebagai data dan Y sebagai label

In [351]:
from sklearn.model_selection import train_test_split
array = dataset.values
X = array[:,0:9]
y = array[:,9]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

Dataset akan terbagi menjadi data latih dan data validasi. Data latih akan digunakan
untuk melatih model dengan mekanisme sebagain diunakan untuk testing dan
sebagainya lagi sebagi data pelatihan sebagaimana telah ditpraktekkan pada
praktikum sebelumnya. Para praktikum ini pelatihan dilakukan menggunakan
mekanisme K-Fold cross validation. Namun demikian sebelum dilakukan K-Fold
validation pastikan data telah teracak sebelumnya.

### K-Fold Cross Validation

k-Fold Cross-Validation Validasi silang adalah prosedur pengambilan sampel
ulang yang digunakan untuk mengevaluasi model pembelajaran mesin pada sampel
data terbatas. Prosedur ini memiliki parameter tunggal yang disebut k yang mengacu pada jumlah grup yang akan dipecah menjadi sampel data tertentu. Karena itu, prosedur ini sering disebut k-fold cross-validation. Ketika nilai spesifik untuk k dipilih, nilai tersebut dapat digunakan sebagai pengganti k dalam referensi ke model, seperti k = 10 menjadi validasi silang 10 kali lipat.
Validasi silang terutama digunakan dalam pembelajaran mesin terapan untuk
memperkirakan kemampuan model pembelajaran mesin pada data yang tidak terlihat.
Yaitu, menggunakan sampel terbatas untuk memperkirakan bagaimana performa
model secara umum saat digunakan untuk membuat prediksi pada data yang tidak
digunakan selama pelatihan model. Prosedur umumnya adalah sebagai berikut:

* Acak kumpulan data berdasarkan id
* Pisahkan kumpulan data menjadi kelompok k
* Untuk setiap grup unik:
  * Ambil kelompok sebagai kelompok data uji
  * Ambil kelompok yang tersisa sebagai kumpulan data pelatihan
  * Catat hasil pengujian masing-masing group

K-Fold ini digunakan untuk validasi model yang digunakan. Evaluasi dilakukan pada dataset latih. Pada praktikum ini digunakan k=10, berarti akan membagi dataset latih menjadi 10 bagian. Satu bagian digunakan untuk testing dan sisanya sebagai data latih.

In [352]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

for name, model in models:
  kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.487778 (0.143540)
NB: 0.662222 (0.125629)
SVM: 0.465556 (0.047778)


### Buat Prediksi
Diantara metode klasifikasi yang telah dievaluasi, Kita pilih algoritma yang mempunyai kinerja paling baik yang akan digunakan untuk membuat metode prediksi Prediksi dilakukan dengan mengevaluasi dataset evaluasi menggunakan metode klasifikasi yang telah terpilih (dalam hal ini metede terbaik adalah SVM). Dengan demikian metode SVM kita gunakan sebagain prediktor

In [353]:
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

### 5. Evaluasi Prediksi
Kami dapat mengevaluasi prediksi dengan membandingkannya dengan hasil
yang diharapkan dalam set validai menggunakan matriks kebingungan dan laporan
klasifikasi.

In [354]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=20, n_informative
=15, n_redundant=5, random_state=1)
from sklearn.metrics import classification_report
print(classification_report(Y_validation, predictions))

              precision    recall  f1-score   support

         1.0       0.25      1.00      0.40         6
         2.0       0.00      0.00      0.00        18

    accuracy                           0.25        24
   macro avg       0.12      0.50      0.20        24
weighted avg       0.06      0.25      0.10        24



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Tugas**
1. Lakukan perbandingan klasifikasi antara SVM, Naive Bayes, KNN untuk dataset iris dan breast cancer
2. Pada kedua dataset lakukan proses normaslisai data dan preproses data untuk
menengani data yang hilang jika ada.

# **Handling Missing Value dan Normalisasi**

In [355]:
def imputasi(df_input) :
  # Mendapatkan daftar kolom pada dataframe
  list_columns = df_input.columns
  # Kolom terakhir merupakan kolom kelas
  class_column = list_columns[-1]
  for column in list_columns[:-1]:
    # Penggantian nilai NaN dilakukan per kolom
    df_input[column] = df_input[column].fillna(df_input.groupby(class_column)[column].transform('mean'))
  return df_input

def minmax(df_input) :
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    df_input[fitur] = (df_input[fitur]-min)/(max-min)
  return df_input

# **Dataset Iris**

In [356]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []

In [357]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-05-08 06:44:12--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.20’

iris.csv.20         100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-05-08 06:44:12 (55.8 MB/s) - ‘iris.csv.20’ saved [3975/3975]



In [358]:
import pandas as pd
import numpy as np
iris_dataset = pd.read_csv('iris.csv')
iris_dataset

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [359]:
iris_clean = imputasi(iris_dataset)
iris_clean = minmax(iris_dataset)

In [360]:
from sklearn.model_selection import train_test_split
array = iris_clean.values
X = array[:,0:4]
y = array[:,4]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

In [361]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

for name, model in models:
  kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.950000 (0.040825)
NB: 0.950000 (0.055277)
SVM: 0.941667 (0.053359)


# **Dataset Breast Cancer**

In [362]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []

In [363]:
! wget https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv

--2024-05-08 06:44:12--  https://raw.githubusercontent.com/frnkldgnwn/utp-ppm/main/breast_cancer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3664 (3.6K) [text/plain]
Saving to: ‘breast_cancer.csv.15’

breast_cancer.csv.1 100%[===================>]   3.58K  --.-KB/s    in 0s      

2024-05-08 06:44:13 (62.9 MB/s) - ‘breast_cancer.csv.15’ saved [3664/3664]



In [364]:
import pandas as pd
import numpy as np
breastcancer_dataset = pd.read_csv('breast_cancer.csv')
breastcancer_dataset

,id,clump_thickness,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nucklei,bland_chromatin,normal_nucleoli,mitoses,diagnosis
0,1000025,5.0,1.0,NaN,1.0,NaN,1.0,3.0,1.0,1.0,benign
1,1002945,5.0,4.0,4.0,5.0,7.0,NaN,3.0,NaN,1.0,benign
2,1015425,3.0,1.0,1.0,1.0,NaN,2.0,3.0,1.0,1.0,benign
3,1016277,6.0,8.0,8.0,1.0,3.0,NaN,3.0,7.0,1.0,benign
4,1017023,4.0,1.0,1.0,3.0,NaN,1.0,3.0,1.0,NaN,benign
...,...,...,...,...,...,...,...,...,...,...,...
95,1167439,2.0,3.0,4.0,4.0,2.0,5.0,2.0,5.0,1.0,malignant
96,1168359,8.0,2.0,3.0,NaN,6.0,3.0,7.0,1.0,1.0,malignant
97,1168736,10.0,10.0,10.0,10.0,10.0,1.0,8.0,8.0,8.0,malignant
98,1169049,7.0,3.0,4.0,4.0,3.0,3.0,3.0,2.0,7.0,malignant


In [365]:
breastcancer_clean = imputasi(breastcancer_dataset)
breastcancer_clean = minmax(breastcancer_dataset)

In [366]:
from sklearn.model_selection import train_test_split
array = breastcancer_clean.values
X = array[:,0:10]
y = array[:,10]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

In [367]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

for name, model in models:
  kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

KNN: 0.925000 (0.100000)
NB: 0.900000 (0.108972)
SVM: 0.950000 (0.082916)
